In [1]:
"""
0=first player (red), 1=second player (blue)
board=np.zero((11,11)) # 1:red, 2:blue, 0:empty
game, node, m(当前步数)
"""
import numpy as np
import torch

import azalea as az

In [2]:
model_path = 'models/hex11-20180712-3362.policy.pth'
config = {
    'game': 'hex',
    'seed': 0xBAD5EED5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

np.random.seed(config['seed'])
torch.manual_seed(config['seed'])
torch.cuda.manual_seed(config['seed'])

policy = az.Policy.load(config, model_path)
config['board_size'] = policy.board_size
print(config)

{'game': 'hex', 'seed': 3134582485, 'device': 'cuda', 'board_size': 11}


In [6]:
gamelib = az.import_game(config['game'])
# new game
game = gamelib.GameState(board_size=config['board_size'])
node = policy.reset(game)
computer_move = None
print(
    # policy.board_size,
    # policy.gamelib,
    # policy.net,
    policy.device,
    policy.simulations,
    policy.search_batch_size,
    policy.exploration_coef,
    policy.exploration_noise_alpha
)

cpu 3000 10 0.75 0.03


In [4]:
def print_board(utils, board, computer_move):
    print('\033[2J')
    if computer_move:
        utils.print_board(board)  # , highlight=computer_move)
        print('last move: {}'.format(utils.format_move(
            board, computer_move)))
    else:
        utils.print_board(board)
        
def ask_move(utils, board, moves):
    while True:
        move_txt = input('Your move? ')
        try:
            move = utils.parse_move(board, moves, move_txt)
            if move in moves:
                return move
        except utils.ParseError:
            print('illegal move:', move_txt)
human = 1

In [ ]:

for m in range(3):
    moves = game.legal_moves()
    if human:
        print_board(gamelib, game.board(), computer_move)
        move = ask_move(gamelib, game.board(), moves)
        action = list(moves).index(move)
    else:
        # computer plays
        action, node, probs, metrics = \
            policy.choose_action(game, node, m,
                                 temperature=0, noise_scale=0)
        computer_move = move = moves[action]
        # print(action, node, probs, metrics, move)
    game.move(move)
    if game.result():
        print_board(gamelib, game.board(), computer_move)
        break
        # return game.result()
    if human:
        node = policy.make_action(game, node, action, moves)
    human = 1 - human
print('end')

In [ ]:
"""
policy and search tree:
 - self.gamelib,
 - self.net,
 - self.device,
 - self.simulations,
 - self.search_batch_size,
 - self.exploration_coef,
 - self.exploration_noise_alpha
 
 tree, 
 *node_id, utils, game, net, device,
 num_simulations, batch_size,
exploration_coef,
 *exploration_noise_scale, exploration_noise_alpha
"""



